In [50]:
from langgraph.graph import START , StateGraph, END
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai  import ChatOpenAI
from langchain.prompts import PromptTemplate
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage
from langchain.chains import LLMChain, ConversationChain
from langchain.memory import ConversationBufferMemory

In [21]:
from typing_extensions import TypedDict
from typing import Annotated

class State(TypedDict):
    messages : Annotated[list[AnyMessage],add_messages]

In [3]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [4]:
chapter_template = """
Template for Story Creation with Agentic AI

Objective:
You are a vital component of Agentic AI, designed to assist users in creating engaging stories.

Remeber : 
The plot of story and chapter should be UNIQUE and more more natural.

Your Role:
- Develop chapter titles based on the user's input.
- Generate story content upon user requests, aligning with the specified chapters.

Your Mission:
Upon receiving this prompt, consider it the beginning of a new story. Your primary task is to create compelling story chapters based on the user input provided below:

{messages}

Now, it’s time to generate chapter titles for the story. Ensure they align with the specified plot of story ({plot}) and adhere to the maximum number of chapters ({max_chapters}).

Your return type (json) :
{chapter_response_formate}
"""


In [5]:
chapter_response_formate = '''
    {
        Story_Name : " ",
        Chapters : {
            chapter_number : chapter name,
            .....
        }
    }
'''

In [77]:
def chapter_maker(messages : State):
    '''

    This function is used to create chapter based on the given zoner of the story to be created.
    It will create chapters and save the chapters created in a file and keep a trace of the chapters created.

    '''

    plot = input("Enter the senario for the story :- ")
    max_chapters = input("Enter the number of chapters for the story :- ")

    chapter_gen_prompt = PromptTemplate(
        input_variables = [messages,plot, max_chapters] ,
        template = chapter_template
    )



    llm_for_chapter = ChatOpenAI(model="gpt-4o-mini")
    chapter_gen_prompt = PromptTemplate(
        input_variables = [messages,plot, max_chapters, chapter_response_formate] ,
        template = chapter_template
    )
    chain = LLMChain(
        llm = llm_for_chapter,
        prompt = chapter_gen_prompt
    )

    respone = {
        "messages": chain.invoke({
            "messages": messages['messages'],
            "plot": plot,
            "max_chapters": max_chapters,
            "chapter_response_formate": chapter_response_formate
    })}
    # chapter = respone['messages'][-1]
    return respone



In [72]:
from langchain_core.messages import HumanMessage
messages = {'messages': HumanMessage(content="Let start")}
plot = "A group of friends stuck in forest and they found that they stuck in a loop, if they dont find way out they each will die on every 65.7 hours, make use this hints to slove this puzzle , like why every one will die at 65.7 hours. "
max_chapters = '12'


In [76]:
messages = {'messages': HumanMessage(content="Let start")}
res = chapter_maker(messages)

In [75]:
print(res['messages']['text'])

```json
{
    "Story_Name": "Shadows of Betrayal",
    "Chapters": {
        "1": "The Unlikely Trio",
        "2": "A Night to Remember",
        "3": "Whispers in the Dark",
        "4": "Crossroads of Deceit",
        "5": "Secrets Unearthed",
        "6": "Escape from Shadows"
    }
}
```
